In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import time
import os
import pickle

In [2]:
os.listdir('./Data/TestData/Test13Feb')

['.ipynb_checkpoints',
 'DS_STRAINING_000-002_T102R100PV000CON-SIMPLIFIED_MAIN.CSV',
 'DS_STRAINING_000-002_T102R097PV000COFF-SIMPLIFIED_MAIN.CSV',
 'DS_STRAINING_000-002_T102R097PV000CON-SIMPLIFIED_MAIN.CSV',
 'Round1Predictions',
 'Round2Predictions']

In [3]:
with open('./ScalersFromTraining/ScalerX.pkl', 'rb') as f:
    scaler_x = pickle.load(f)

In [4]:
with open('./ScalersFromTraining/ScalerY.pkl', 'rb') as f:
    scaler_y = pickle.load(f)

In [5]:
def get_labels(file_path):
    '''
    file_path: string 
    '''
    controls = file_path.split("_")[3] #eg. 'T102R009PV680COF'
    regulator = int(controls[5]) + int(controls[7]) / 10
    phase = int(controls[10:13]) / 10
    capacitor = 0
    if controls[14:16] == 'ON':
        capacitor = 1
#     return [transformer, regulator, phase, capacitor]
    return [regulator, phase, capacitor]

def sequencify(df, filename):
    # takes in raw dataframe
    # get control settings
    # each row is a 2D array:
    #   [ [x_t], [x_{t+1}] ]
    #   [y (ie. control settings) ]
    data_x = []
    data_y = []
    labels = get_labels(filename)
    for i in range(1, len(df)):
        data_x.append([df.iloc[i-1,1:].values,
#                        labels,
                       df.iloc[i,1:].values])
        data_y.append(labels)
    
    return np.stack(data_x), np.array(data_y)

In [8]:
model_a = tf.keras.models.load_model('./Saved_Models/Training/MultiheadedAttentionLSTM_Simple.h5', compile=False)
model_b = tf.keras.models.load_model('./Saved_Models/Training/B_LSTM.h5', compile=False)
model_c = tf.keras.models.load_model('./Saved_Models/Training/C_LSTM.h5', compile=False)

In [10]:
def predict(model, model_type, x_scaled, y_scaled, i):
    start_time = time.time()
    if model_type == 'A':
        pred = model.predict([x_scaled, x_scaled, y_scaled])
    elif model_type == 'B':
        pred = model.predict([x_scaled, y_scaled])
    else:
        pred = model.predict(x_scaled)
    end_time = time.time()
    pred_time = end_time - start_time
    print(f"Prediction Time for Model {model_type}: {pred_time}s")
    np.savetxt(f"./Data/TestData/Test13Feb/Round2Predictions/Test{i}_Predictions{model_type}_{file_path}.csv",
               pred, delimiter=',')

def get_predictions(file_path, index, model_a, model_b, model_c):
    test = pd.read_csv(f"./Data/TestData/Test13Feb/{file_path}", skiprows=[0,1,2,4])
    x, y = sequencify(test, file_path)
    x_scaled = scaler_x.transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    y_scaled = scaler_y.transform(y)
    print(f"Predictions for {file_path}:")
    predict(model_a, "A", x_scaled, y_scaled, index)
    predict(model_b, "B", x_scaled, y_scaled, index)
    predict(model_c, "C", x_scaled, y_scaled, index) 
    print("**********\n")

In [11]:
i = 1
for file_path in os.listdir('./Data/TestData/Test13Feb'):
    if file_path[0] == 'D':
        get_predictions(file_path, i, model_a, model_b, model_c)
        i += 1    

Predictions for DS_STRAINING_000-002_T102R100PV000CON-SIMPLIFIED_MAIN.CSV:
Prediction Time for Model A: 4.945444345474243s
Prediction Time for Model B: 0.49169921875s
Prediction Time for Model C: 1.0756793022155762s
**********

Predictions for DS_STRAINING_000-002_T102R097PV000COFF-SIMPLIFIED_MAIN.CSV:
Prediction Time for Model A: 0.2127683162689209s
Prediction Time for Model B: 0.17661333084106445s
Prediction Time for Model C: 0.2557835578918457s
**********

Predictions for DS_STRAINING_000-002_T102R097PV000CON-SIMPLIFIED_MAIN.CSV:
Prediction Time for Model A: 0.21486258506774902s
Prediction Time for Model B: 0.17293334007263184s
Prediction Time for Model C: 0.25119829177856445s
**********

